# Controller Usage

Here we go through a rocket trajectory simulation with canards and roll control. To use controllers and canards, a few things must be added to the simulation.

Let's start by importing the rocketpy module and other necessary modules.

In [ ]:
from rocketpy import Environment, SolidMotor, Rocket, Flight, Function
from simple_pid import *
import numpy as np


If you are using Jupyter Notebooks, it is recommended to run the following line to make matplotlib plots which will be shown later interactive and higher quality.

In [ ]:
%matplotlib widget

### Creating a simple Environment

In [ ]:
Env = Environment(
    latitude=32.990254, longitude=-106.974998, elevation=1400
)

Env.set_atmospheric_model(type="custom_atmosphere")

In [ ]:
Env.info()

### Creating a Motor

In [ ]:
Pro75M1670 = SolidMotor(
    thrust_source="../../data/motors/Cesaroni_M1670.eng",
    dry_mass=1.815,
    dry_inertia=(0.125, 0.125, 0.002),
    center_of_dry_mass=0.317,
    grains_center_of_mass_position=0.397,
    burn_time=3.9,
    grain_number=5,
    grain_separation=5 / 1000,
    grain_density=1815,
    grain_outer_radius=33 / 1000,
    grain_initial_inner_radius=15 / 1000,
    grain_initial_height=120 / 1000,
    nozzle_radius=33 / 1000,
    throat_radius=11 / 1000,
    interpolation_method="linear",
    nozzle_position=0,
    coordinate_system_orientation="nozzle_to_combustion_chamber",
)

In [ ]:
Pro75M1670.info()

### Creating a Rocket

In [ ]:
calisto = Rocket(
    radius=127 / 2000,
    mass=19.197 - 2.956 - 1.815,
    inertia=(6.321, 6.321, 0.034),
    power_off_drag="../../data/calisto/powerOffDragCurve.csv",
    power_on_drag="../../data/calisto/powerOnDragCurve.csv",
    center_of_mass_without_motor=0,
    coordinate_system_orientation="tail_to_nose",
)

rail_buttons = calisto.set_rail_buttons(
    upper_button_position=0.2 - 0.1182359460624346,
    lower_button_position=-0.5 - 0.1182359460624346,
    angular_position=45,
)

#### Adding Aerodynamic Surfaces

Now we define the aerodynamic surfaces. 
It is important to define the canards as shown bellow so their cant angle can be changed during the simulation

In [ ]:
calisto.add_motor(Pro75M1670, position=-1.255)

NoseCone = calisto.add_nose(
    length=0.55829, kind="vonKarman", position=0.71971 + 0.55829
)

fin_set = calisto.add_trapezoidal_fins(
    n=4,
    root_chord=0.130,
    tip_chord=0.080,
    span=0.130,
    position=-1.04956,
    cant_angle=0.5,
    radius=None,
    airfoil=None,
)

calisto.canards = calisto.add_trapezoidal_fins(
    2, span=0.07, root_chord=0.07, tip_chord=0.07, position=0.7, cant_angle=0, name="Canards"
)

Tail = calisto.add_tail(
    top_radius=0.0635, bottom_radius=0.0435, length=0.060, position=-1.194656
)

Always important to check the static margin to guarantee that the rocket is stable.

In [ ]:
calisto.static_margin()

### Defining controller

Here we add the controller to the rocket. Much like adding parachutes, the method call
`Calisto.addControllers` requires a function that defines what the controller will do during flight

In this case, the controller function uses a PID to calculate the cant angle of the 
canards given the roll speed of the rocket (omega3). It activates only after the motor burn out.

Here there are a few global variables used for checking how the cantAngle and omega3 behaved during the flight simulation

In [ ]:
#TODO: better way to save cantAngleList and omegaList

cantAngleList = []
omegaList = []

# defines PID controller
pid = PID(
    Kp=0.015,
    Ki=0.0315,
    Kd=0.000,
    sample_time=None,
    output_limits=(-np.radians(8), np.radians(8)),
)

# TODO: make controller attributes accessible in the controllerFunction

def controllerFunction(flightClassInstance):
    global cantAngleList
    global omegaList
    global pid

    rocket = flightClassInstance.rocket
    omega3 = flightClassInstance.y_sol[-1]
    time = flightClassInstance.t
    servo_speed = 60/0.8 # degrees per second

    # Checks if the motor has fineshed burning
    if time < rocket.motor.burn_out_time:
        return

    # PID controller receives omega and returns delta
    cantAngleRad = pid(omega3, dt=1 / 100) 

    cantAngle = cantAngleRad*180/np.pi

    rocket.canards = rocket.changeCantAngle(cantAngle, rocket.canards)
    cantAngleList.append([time, cantAngle])
    omegaList.append([time, omega3])

    
# Clear controller list so that this cell can be rerun without rerunning the Rocket class
# cells above.
calisto.controllers = []
    
calisto.addController(name='CanardPID', controllerFunction=controllerFunction, samplingRate=100, lag=0)

The last

In [ ]:
# cantAngleList = []
# omegaList = []

# # defines PID controller
# pid = PID(
#     Kp=0.015,
#     Ki=0.0315,
#     Kd=0.000,
#     sample_time=None,
#     output_limits=(-np.radians(8), np.radians(8)),
# )

# # TODO: make controller attributes accessible in the controllerFunction

# def controllerFunction(flightClassInstance):
#     global cantAngleList
#     global omegaList
#     global pid

#     rocket = flightClassInstance.rocket
#     omega3 = flightClassInstance.y[-1]
#     time = flightClassInstance.t
#     servo_speed = 60/0.8 # degrees per second

#     # Checks if the motor has fineshed burning
#     if time < rocket.motor.burnOutTime:
#         return

#     # PID controller receives omega and returns delta
#     cantAngleRad = pid(omega3, dt=1 / 100) 

#     cantAngle = cantAngleRad*180/np.pi

#     maximumCantAngleChange = servo_speed*1/100 # degrees per second

#     rocket.canards = rocket.changeCantAngle(cantAngle, rocket.canards, maximumCantAngleChange)
#     cantAngleList.append([time, cantAngle])
#     omegaList.append([time, omega3])

    


# # Clear controller list so that this cell can be rerun without rerunning the Rocket class
# # cells above.
# Calisto.controllers = []
    
# Calisto.addController(name='CanardPID', controllerFunction=controllerFunction, samplingRate=100, lag=0)

## Simulating a Flight

We can then simulate the flight and check the relevant information

In [ ]:
TestFlight = Flight(rocket=calisto, environment=Env, rail_length=5.2, inclination=85, heading=0, time_overshoot=False, terminate_on_apogee=True, equations_of_motion="solid_propulsion")

In [ ]:
Function(cantAngleList,'time','cant Angle',interpolation='linear')()
Function(omegaList,'time','omega',interpolation='linear')()

In [ ]:
TestFlight.info()

In [ ]:
TestFlight.plots.all()